# Problem Set 5



Team: 
    Lea Schmierer 3546563;
    Angelina Basova 3704658;
    Daniel Knorr 3727033

## Exercise 1


a) 

relevant book sections: 6.1. Changing α, Check for Important Mathematical Properties Associated with the Power Method.

Answer: The main advantage of making the dumping factor smaller is a faster convergence of the pageranks as less iterations are needed. The main disadvantage is that it places less emphasis on the hyperlink structure of the web and hence much more on the teleportation tendencies of the surfers. 



b)

relevant book sections: 5.1.1. Check for Important Mathematical Properties Associated with the Power Method.

Answer: We would need to change the parameter beta, which is called alpha in the book. Specifically, a smaller alpha results into a faster convergence and hence less iterations needed.

Assuming that the tolerance parameter $\tau$ is $10^{-6}$ and $\alpha = 0.987$, then the number of iterations equals to $\frac{log_{10} 10^{-6}}{log_{10}0.987} = 1088$.

To reduce the number of iterations with the same tolerance parameter $\tau$, $\alpha$ should be reduced to $0.87$. Then the number of iterations equals to $\frac{log_{10} 10^{-6}}{log_{10}0.87} = 101$. Hence in this scenario $\alpha$ should be multiplied with the factor 0.88, which is the reduced of $87 / 98.7$

c)

relevant book sections: 6.2. The Personalization Vector vT

Answer: The last part of the google matrix, in other words the teleportation matrix, needs to be adjusted. Hence, instead of $1/n*e^T$ which assigns the same probability to each web page, we will use $ev^T$. The matrix $v^T$ is a 1xN matrix, where preferred webpages are assigned a higher probability than less preferred pages. The most common use case of the matrix $v^T$ is to personalize the web experience.

## Exercise 2

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType


######################
# Solution of 4a) from Problem set 4
print("Exercise 4a)")
######################

sparkSession = SparkSession.builder.appName('A4E4').getOrCreate()

print("# --------------------------------")
print("# Create DataFrame DF1 webStanfordDF")
print("# --------------------------------")

rdd_datafile = sparkSession.sparkContext.textFile('./web-Stanford_small.txt')

rdd_web_standford = rdd_datafile.map(lambda x: x.split("\t"))

webStanfordRDD = rdd_web_standford.map(lambda p: (
    int(p[0]), int(p[1].strip())))

schema = StructType([
    StructField("FromNodeId", IntegerType(), True),
    StructField("ToNodeId", IntegerType(), True)
])

webStanfordDF = sparkSession.createDataFrame(webStanfordRDD, schema)

# Get all Nodes as a Set
listFromNodeId = webStanfordDF.select(
    webStanfordDF["FromNodeId"]).toPandas()['FromNodeId']
listeToNodeId = webStanfordDF.select(
    webStanfordDF["ToNodeId"]).toPandas()['ToNodeId']

listFromNodeId = list(listFromNodeId)
listeToNodeId = list(listeToNodeId)

allNodesList = listFromNodeId + listeToNodeId
allNodesSet = set(allNodesList)


# Define dictionary
dic = {}


def generateDictionary(allNodesSet, webStanfordDF, dic):
    for node in allNodesSet:
        liste = []
        filteredByFromNodeId = webStanfordDF.filter(
            webStanfordDF["FromNodeId"] == node)
        listOutgoingLinksTo = filteredByFromNodeId.select(
            filteredByFromNodeId["ToNodeId"]).toPandas()['ToNodeId']
        filteredByToNodeId = webStanfordDF.filter(
            webStanfordDF["ToNodeId"] == node)
        listIngoingLinksFrom = filteredByToNodeId.select(filteredByToNodeId["FromNodeId"]).toPandas()[
            'FromNodeId']
        listOutgoingLinksTo = list(listOutgoingLinksTo)
        listIngoingLinksFrom = list(listIngoingLinksFrom)
        liste.append(listOutgoingLinksTo)
        liste.append(listIngoingLinksFrom)
        dic[node] = liste


generateDictionary(allNodesSet, webStanfordDF, dic)

print(
    "Dictionary Composition: Node i: <[Out-neighbors of i]>,<[In-neighbors of i]>")
print(dic)


Exercise 4a)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/28 14:29:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
# --------------------------------
# Create DataFrame DF1 webStanfordDF
# --------------------------------


Dictionary Composition: Node i: <[Out-neighbors of i]>,<[In-neighbors of i]>
{1: [[6548, 15409], []], 2: [[17794, 25202, 53625, 54582, 64930, 73764, 84477, 98628, 100193, 102355, 105318, 105730, 115926, 140864, 163550, 164599, 175799, 178642, 181714, 190453, 204189, 204604, 210870, 213966, 225119, 241596, 243294, 246897, 251658, 252915, 280935], [252915, 246897, 251658, 280935, 213966, 243294, 225119, 241596, 178642, 210870, 204189, 190453, 204604, 181714, 164599, 175799, 163550, 140864, 115926, 73764, 105730, 98628, 84477, 105318, 100193, 102355, 64930, 54582, 53625, 25202, 17794]], 3: [[74361], []], 4: [[35716, 96512, 186750, 225872], []], 5: [[6545, 14395, 59745, 67503, 134375, 170452, 220513], [59745, 6545, 14395]], 6: [[6540, 59742], [59742, 6540]], 7: [[45366, 123814, 149627, 194146, 198520, 240934], [45366]], 8: [[35084, 41117, 58764, 86240, 222332, 248139, 270771], []], 9: [[32423], []], 10: [[6536, 55087, 118572], [118572, 6536]], 11: [[149993], [149993]], 12: [[68889, 95163, 

In [2]:
N = len(dic)

print(f" Number of nodes: {N}")

print("ok")

 Number of nodes: 1587
ok


In [3]:
import numpy as np

In [12]:
#import Blatt4_Aufg4

# "Dictionary Composition: Node i: <[Out-neighbors of i]>,<[In-neighbors of i]>"
dataStructure = dic
N = len(dataStructure)
r_old = {} # node : pagerank 
r_new = {}  # node : pagerank
beta = 0.8


def initiate_R_Dic_Old_And_New(dataStructure):
    for key in dataStructure.keys():
        r_old[key] = 1 / N
        r_new[key] = 0

initiate_R_Dic_Old_And_New(dataStructure)


# count == 0 --> 1. Iteration!
# maybe replace range from 6 to 5
for T in range(6):

    # A bullet point in formula
    for key in dataStructure.keys():
        ingoingListOfCurrentKey = dataStructure[key][1]
        r_j = 0

        for item in ingoingListOfCurrentKey:
            d = len(dataStructure[item][0])  # amountOutgoingsFromCurrentItem
            r_j += beta * (r_old[item] * (1 / d))

        r_new[key] = r_j


    # B bullet point in formula
    S = np.sum(list(r_new.values()))
    print(f" Iteration: {T}, S = {S}")

    for key in dataStructure.keys():
        r_new[key] += (1 - beta * S) / N 


    # last bullet point in formula
    for element in r_old.keys():
        r_old[element] = r_new[element]



print(f"Final Pagerank {r_new}")

 Iteration: 0, S = 0.0998109640831758
 Iteration: 1, S = 0.1439154420732032
 Iteration: 2, S = 0.15936755548143244
 Iteration: 3, S = 0.16728662511812775
 Iteration: 4, S = 0.17280427248465796
 Iteration: 5, S = 0.1766063379605863
Final Pagerank {1: 0.0005410932133784064, 2: 0.02676168105671601, 3: 0.0005410932133784064, 4: 0.0005410932133784064, 5: 0.0007270296747387574, 6: 0.0014161396216078891, 7: 0.0010930198211372617, 8: 0.0005410932133784064, 9: 0.0005410932133784064, 10: 0.0009693698114331588, 11: 0.0007210722741443603, 12: 0.0005410932133784064, 13: 0.0021898509357457505, 14: 0.0005410932133784064, 15: 0.0005538678967209461, 16: 0.0008728233278955918, 17: 0.0005410932133784064, 18: 0.0014161396216078891, 19: 0.0005609860146395265, 20: 0.0006302762627725026, 21: 0.0006112086756286028, 22: 0.0007470515555644626, 23: 0.0010142003394318908, 24: 0.0005410932133784064, 25: 0.0006167306078120654, 213021: 0.0005456425715164767, 237599: 0.0005755886263633348, 131107: 0.00054564257151647

## Exercise 3

You can find the solution of this exercise in the zip file as a pdf. 

## Exercise 4

You can find the solution of this exercise in the zip file as a pdf. 

## Exercise 5

You can find the solution of this exercise in the zip file as a pdf. 